<a href="https://colab.research.google.com/github/AKerby/dsci_325_module_7_more_data_management_in_python/blob/main/Table%20Joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab Prep

Execute the following code cells to whenever you open/restart the notebook in Google Colab.

In [ ]:
!pip install "polars[all]"

In [ ]:
!wget https://github.com/WSU-DataScience/dsci_325_module_7_more_data_management_in_python/raw/main/sample_data.zip

In [ ]:
!unzip ./sample_data.zip

In [ ]:
!pip install more_polars

# Joining Tables with `dfply`

In [ ]:
import polars as pl

## Example

The following tables were taken from the Wikipedia page for joins and will provide a small but sufficiently complicated example for introducing joins.

In [ ]:
dept = pl.read_csv("./sample_data/department.csv")
dept

shape: (4, 2)
┌────────┬─────────────┐
│ DeptID ┆ DeptName    │
│ ---    ┆ ---         │
│ i64    ┆ str         │
╞════════╪═════════════╡
│ 31     ┆ Sales       │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 34     ┆ Clerical    │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 35     ┆ Marketing   │
└────────┴─────────────┘

In [ ]:
empl = pl.read_csv("./sample_data/employee.csv")
empl

shape: (6, 3)
┌────────────┬────────────┬────────┐
│ LastName   ┆ EmployeeID ┆ DeptID │
│ ---        ┆ ---        ┆ ---    │
│ str        ┆ i64        ┆ i64    │
╞════════════╪════════════╪════════╡
│ Rafferty   ┆ 102        ┆ 31     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ Jones      ┆ 105        ┆ 33     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ Heisenberg ┆ 106        ┆ 33     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ Robinson   ┆ 108        ┆ 34     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ Smith      ┆ 110        ┆ 34     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ Williams   ┆ 111        ┆ null   │
└────────────┴────────────┴────────┘

In [ ]:
salary = pl.read_csv("./sample_data/salary.csv")
salary

shape: (18, 3)
┌────────────┬──────┬────────┐
│ EmployeeID ┆ Year ┆ Salary │
│ ---        ┆ ---  ┆ ---    │
│ i64        ┆ i64  ┆ i64    │
╞════════════╪══════╪════════╡
│ 102        ┆ 2020 ┆ 52000  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 102        ┆ 2021 ┆ 53000  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 102        ┆ 2022 ┆ 54000  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 105        ┆ 2020 ┆ 73000  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ ...        ┆ ...  ┆ ...    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 110        ┆ 2022 ┆ 61000  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 111        ┆ 2020 ┆ 72000  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 111        ┆ 2021 ┆ 73000  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 111        ┆ 2022 ┆ 74000  │
└────────────┴──────┴────────┘

## LEFT JOIN

In [ ]:
empl.join(dept, on='DeptID', how='left')

shape: (6, 4)
┌────────────┬────────────┬────────┬─────────────┐
│ LastName   ┆ EmployeeID ┆ DeptID ┆ DeptName    │
│ ---        ┆ ---        ┆ ---    ┆ ---         │
│ str        ┆ i64        ┆ i64    ┆ str         │
╞════════════╪════════════╪════════╪═════════════╡
│ Rafferty   ┆ 102        ┆ 31     ┆ Sales       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Jones      ┆ 105        ┆ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Heisenberg ┆ 106        ┆ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Robinson   ┆ 108        ┆ 34     ┆ Clerical    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Smith      ┆ 110        ┆ 34     ┆ Clerical    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Williams   ┆ 111        ┆ null   ┆ null        │
└────────────┴────────────┴────────┴─────────────┘

## When the index names differ

In [ ]:
empl2 = empl.rename({'DeptID':'dept_id'})
empl2

shape: (6, 3)
┌────────────┬────────────┬─────────┐
│ LastName   ┆ EmployeeID ┆ dept_id │
│ ---        ┆ ---        ┆ ---     │
│ str        ┆ i64        ┆ i64     │
╞════════════╪════════════╪═════════╡
│ Rafferty   ┆ 102        ┆ 31      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ Jones      ┆ 105        ┆ 33      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ Heisenberg ┆ 106        ┆ 33      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ Robinson   ┆ 108        ┆ 34      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ Smith      ┆ 110        ┆ 34      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ Williams   ┆ 111        ┆ null    │
└────────────┴────────────┴─────────┘

In [ ]:
empl2.join(dept, left_on='dept_id', right_on='DeptID', how='left')

shape: (6, 4)
┌────────────┬────────────┬─────────┬─────────────┐
│ LastName   ┆ EmployeeID ┆ dept_id ┆ DeptName    │
│ ---        ┆ ---        ┆ ---     ┆ ---         │
│ str        ┆ i64        ┆ i64     ┆ str         │
╞════════════╪════════════╪═════════╪═════════════╡
│ Rafferty   ┆ 102        ┆ 31      ┆ Sales       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Jones      ┆ 105        ┆ 33      ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Heisenberg ┆ 106        ┆ 33      ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Robinson   ┆ 108        ┆ 34      ┆ Clerical    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Smith      ┆ 110        ┆ 34      ┆ Clerical    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Williams   ┆ 111        ┆ null    ┆ null        │
└────────────┴────────────┴─────────┴─────────────┘

## No RIGHT JOIN, use a LEFT JOIN

In [ ]:
dept.join(empl, on='DeptID', how='left')

shape: (6, 4)
┌────────┬─────────────┬────────────┬────────────┐
│ DeptID ┆ DeptName    ┆ LastName   ┆ EmployeeID │
│ ---    ┆ ---         ┆ ---        ┆ ---        │
│ i64    ┆ str         ┆ str        ┆ i64        │
╞════════╪═════════════╪════════════╪════════════╡
│ 31     ┆ Sales       ┆ Rafferty   ┆ 102        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 33     ┆ Engineering ┆ Jones      ┆ 105        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 33     ┆ Engineering ┆ Heisenberg ┆ 106        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 34     ┆ Clerical    ┆ Robinson   ┆ 108        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 34     ┆ Clerical    ┆ Smith      ┆ 110        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 35     ┆ Marketing   ┆ null       ┆ null       │
└────────┴─────────────┴────────────┴────────────┘

## Full Outer Join

In [ ]:
empl.join(dept, on='DeptID', how='outer')

shape: (7, 4)
┌────────────┬────────────┬────────┬─────────────┐
│ LastName   ┆ EmployeeID ┆ DeptID ┆ DeptName    │
│ ---        ┆ ---        ┆ ---    ┆ ---         │
│ str        ┆ i64        ┆ i64    ┆ str         │
╞════════════╪════════════╪════════╪═════════════╡
│ Rafferty   ┆ 102        ┆ 31     ┆ Sales       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Jones      ┆ 105        ┆ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Heisenberg ┆ 106        ┆ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Robinson   ┆ 108        ┆ 34     ┆ Clerical    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Smith      ┆ 110        ┆ 34     ┆ Clerical    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Williams   ┆ 111        ┆ null   ┆ null        │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ null       ┆ null       ┆ 35     ┆ Marketing   │
└────────────┴────────────┴────────┴─────────────┘

## Inner Join

In [ ]:
empl.join(dept, on='DeptID', how='inner')

shape: (5, 4)
┌────────────┬────────────┬────────┬─────────────┐
│ LastName   ┆ EmployeeID ┆ DeptID ┆ DeptName    │
│ ---        ┆ ---        ┆ ---    ┆ ---         │
│ str        ┆ i64        ┆ i64    ┆ str         │
╞════════════╪════════════╪════════╪═════════════╡
│ Rafferty   ┆ 102        ┆ 31     ┆ Sales       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Jones      ┆ 105        ┆ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Heisenberg ┆ 106        ┆ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Robinson   ┆ 108        ┆ 34     ┆ Clerical    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Smith      ┆ 110        ┆ 34     ┆ Clerical    │
└────────────┴────────────┴────────┴─────────────┘

## Chaining multiple joins

To join more than two tables, simply dot-chain each join onto the first table.

```{python}
(table1
 .join(table2, ...)
 .join(table3, ...)
)
```

### Example - Average salary by department

Suppose we want to compute the average salary for each department, sorted from largest to smallest.  Before performing the query, make sure you understand the relationships between tables.

<img src="https://github.com/WSU-DataScience/dsci_325_module_7_more_data_management_in_python/raw/main/img/table_relationship.png" width="800">


#### Edit 1 - Joining the salary and employee tables

**Notes.**

1. We can't perform the aggregation until *after* joining the departments and salaries.
2. Since we are primarily interested in aggregating salaries, we need to use a *left* join with the salary table on the left.
3. To illustrate the coding process, we will show each successive edit in a separate cell.  In practice, all the code would be developed in one cell, but rerun to check the results at each step.

In [ ]:
(salary
 .join(empl, on='EmployeeID', how='left')
).head()

shape: (5, 5)
┌────────────┬──────┬────────┬──────────┬────────┐
│ EmployeeID ┆ Year ┆ Salary ┆ LastName ┆ DeptID │
│ ---        ┆ ---  ┆ ---    ┆ ---      ┆ ---    │
│ i64        ┆ i64  ┆ i64    ┆ str      ┆ i64    │
╞════════════╪══════╪════════╪══════════╪════════╡
│ 102        ┆ 2020 ┆ 52000  ┆ Rafferty ┆ 31     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 102        ┆ 2021 ┆ 53000  ┆ Rafferty ┆ 31     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 102        ┆ 2022 ┆ 54000  ┆ Rafferty ┆ 31     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 105        ┆ 2020 ┆ 73000  ┆ Jones    ┆ 33     │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 105        ┆ 2021 ┆ 74000  ┆ Jones    ┆ 33     │
└────────────┴──────┴────────┴──────────┴────────┘

#### Edit 2  - Joining on the departments

In [ ]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
).head()

shape: (5, 6)
┌────────────┬──────┬────────┬──────────┬────────┬─────────────┐
│ EmployeeID ┆ Year ┆ Salary ┆ LastName ┆ DeptID ┆ DeptName    │
│ ---        ┆ ---  ┆ ---    ┆ ---      ┆ ---    ┆ ---         │
│ i64        ┆ i64  ┆ i64    ┆ str      ┆ i64    ┆ str         │
╞════════════╪══════╪════════╪══════════╪════════╪═════════════╡
│ 102        ┆ 2020 ┆ 52000  ┆ Rafferty ┆ 31     ┆ Sales       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 102        ┆ 2021 ┆ 53000  ┆ Rafferty ┆ 31     ┆ Sales       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 102        ┆ 2022 ┆ 54000  ┆ Rafferty ┆ 31     ┆ Sales       │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 105        ┆ 2020 ┆ 73000  ┆ Jones    ┆ 33     ┆ Engineering │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 105        ┆ 2021 ┆ 74000  ┆ Jones    ┆ 33     ┆ Engineering │
└────────────┴──────┴────────┴──────────┴────────┴─────────────┘

#### Edit 3  - Group and aggregate

In [ ]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
 .groupby('DeptName')
 .agg([pl.col('Salary').mean().alias('Average Salary'),
      ])
)

shape: (4, 2)
┌─────────────┬────────────────┐
│ DeptName    ┆ Average Salary │
│ ---         ┆ ---            │
│ str         ┆ f64            │
╞═════════════╪════════════════╡
│ Clerical    ┆ 77000.0        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ null        ┆ 73000.0        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Engineering ┆ 79000.0        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Sales       ┆ 53000.0        │
└─────────────┴────────────────┘

#### Edit 4  - Sort the results from largest to smallest

In [ ]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
 .groupby('DeptName')
 .agg([pl.col('Salary').mean().alias('Average Salary'),
      ])
 .sort('Average Salary', reverse=True)
)

shape: (4, 2)
┌─────────────┬────────────────┐
│ DeptName    ┆ Average Salary │
│ ---         ┆ ---            │
│ str         ┆ f64            │
╞═════════════╪════════════════╡
│ Engineering ┆ 79000.0        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Clerical    ┆ 77000.0        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ null        ┆ 73000.0        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Sales       ┆ 53000.0        │
└─────────────┴────────────────┘

## Example 2 - Joining Batting to People

As an example of a joining on more than 1 column, consider the following task.

**Task:** Create a table with the total runs score for each team in 2009.  Include the teams proper name and the name of their home park.

In [ ]:
batting = pl.read_csv("./sample_data/baseball/core/Batting.csv")
batting.head()

shape: (5, 22)
┌───────────┬────────┬───────┬────────┬─────┬──────┬──────┬──────┬──────┐
│ playerID  ┆ yearID ┆ stint ┆ teamID ┆ ... ┆ HBP  ┆ SH   ┆ SF   ┆ GIDP │
│ ---       ┆ ---    ┆ ---   ┆ ---    ┆     ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ str       ┆ i64    ┆ i64   ┆ str    ┆     ┆ str  ┆ str  ┆ str  ┆ i64  │
╞═══════════╪════════╪═══════╪════════╪═════╪══════╪══════╪══════╪══════╡
│ abercda01 ┆ 1871   ┆ 1     ┆ TRO    ┆ ... ┆ null ┆ null ┆ null ┆ 0    │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ addybo01  ┆ 1871   ┆ 1     ┆ RC1    ┆ ... ┆ null ┆ null ┆ null ┆ 0    │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ allisar01 ┆ 1871   ┆ 1     ┆ CL1    ┆ ... ┆ null ┆ null ┆ null ┆ 1    │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ allisdo01 ┆ 1871   ┆ 1     ┆ WS3    ┆ ... ┆ null ┆ null ┆ null ┆ 0    │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ ansonca01 ┆ 1871   ┆ 1     ┆ RC1    ┆ ... ┆ null ┆ null ┆ null ┆ 0    │
└───────────┴────────┴───────┴────────┴─────┴──────┴──────┴──────┴──────┘

In [ ]:
teams = pl.read_csv("./sample_data/baseball/core/Teams.csv")
teams.head()

shape: (5, 48)
┌────────┬──────┬────────┬──────────┬─────┬─────┬──────────┬────────────────┬─────────────┐
│ yearID ┆ lgID ┆ teamID ┆ franchID ┆ ... ┆ PPF ┆ teamIDBR ┆ teamIDlahman45 ┆ teamIDretro │
│ ---    ┆ ---  ┆ ---    ┆ ---      ┆     ┆ --- ┆ ---      ┆ ---            ┆ ---         │
│ i64    ┆ str  ┆ str    ┆ str      ┆     ┆ i64 ┆ str      ┆ str            ┆ str         │
╞════════╪══════╪════════╪══════════╪═════╪═════╪══════════╪════════════════╪═════════════╡
│ 1871   ┆ NA   ┆ BS1    ┆ BNA      ┆ ... ┆ 98  ┆ BOS      ┆ BS1            ┆ BS1         │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ NA   ┆ CH1    ┆ CNA      ┆ ... ┆ 102 ┆ CHI      ┆ CH1            ┆ CH1         │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ NA   ┆ CL1    ┆ CFC      ┆ ... ┆ 100 ┆ CLE      ┆ CL1            ┆ CL1         │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ NA   ┆ FW1    ┆ KEK      ┆ ... ┆ 107 ┆ KEK      ┆ FW1            ┆ FW1         │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ NA   ┆ NY2    ┆ NNA      ┆ ... ┆ 88  ┆ NYU      ┆ NY2            ┆ NY2         │
└────────┴──────┴────────┴──────────┴─────┴─────┴──────────┴────────────────┴─────────────┘

## Things we need from `Batting.csv`

* Year
* Runs
* Team

In [ ]:
batting.head(2)

shape: (2, 22)
┌───────────┬────────┬───────┬────────┬─────┬──────┬──────┬──────┬──────┐
│ playerID  ┆ yearID ┆ stint ┆ teamID ┆ ... ┆ HBP  ┆ SH   ┆ SF   ┆ GIDP │
│ ---       ┆ ---    ┆ ---   ┆ ---    ┆     ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ str       ┆ i64    ┆ i64   ┆ str    ┆     ┆ str  ┆ str  ┆ str  ┆ i64  │
╞═══════════╪════════╪═══════╪════════╪═════╪══════╪══════╪══════╪══════╡
│ abercda01 ┆ 1871   ┆ 1     ┆ TRO    ┆ ... ┆ null ┆ null ┆ null ┆ 0    │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ addybo01  ┆ 1871   ┆ 1     ┆ RC1    ┆ ... ┆ null ┆ null ┆ null ┆ 0    │
└───────────┴────────┴───────┴────────┴─────┴──────┴──────┴──────┴──────┘

#### Selecting down to the necessary columns

In [ ]:
batting_select = (batting
                  .select(['teamID',
                           'yearID',
                           'R'])
                 )
batting_select.head()

shape: (5, 3)
┌────────┬────────┬─────┐
│ teamID ┆ yearID ┆ R   │
│ ---    ┆ ---    ┆ --- │
│ str    ┆ i64    ┆ i64 │
╞════════╪════════╪═════╡
│ TRO    ┆ 1871   ┆ 0   │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┤
│ RC1    ┆ 1871   ┆ 30  │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┤
│ CL1    ┆ 1871   ┆ 28  │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┤
│ WS3    ┆ 1871   ┆ 28  │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┤
│ RC1    ┆ 1871   ┆ 29  │
└────────┴────────┴─────┘

In [ ]:
batting_select.shape

(107429, 3)

## Things we need from `Teams.csv`

* Year
* Runs
* Team

In [ ]:
teams.head(2)

shape: (2, 48)
┌────────┬──────┬────────┬──────────┬─────┬─────┬──────────┬────────────────┬─────────────┐
│ yearID ┆ lgID ┆ teamID ┆ franchID ┆ ... ┆ PPF ┆ teamIDBR ┆ teamIDlahman45 ┆ teamIDretro │
│ ---    ┆ ---  ┆ ---    ┆ ---      ┆     ┆ --- ┆ ---      ┆ ---            ┆ ---         │
│ i64    ┆ str  ┆ str    ┆ str      ┆     ┆ i64 ┆ str      ┆ str            ┆ str         │
╞════════╪══════╪════════╪══════════╪═════╪═════╪══════════╪════════════════╪═════════════╡
│ 1871   ┆ NA   ┆ BS1    ┆ BNA      ┆ ... ┆ 98  ┆ BOS      ┆ BS1            ┆ BS1         │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ NA   ┆ CH1    ┆ CNA      ┆ ... ┆ 102 ┆ CHI      ┆ CH1            ┆ CH1         │
└────────┴──────┴────────┴──────────┴─────┴─────┴──────────┴────────────────┴─────────────┘

#### Selecting down to the necessary columns

In [ ]:
teams_select = (teams.select(['yearID',
                              'teamID',
                              'name',
                              'park',
                             ]))
teams_select.head()

shape: (5, 4)
┌────────┬────────┬─────────────────────────┬──────────────────────────────┐
│ yearID ┆ teamID ┆ name                    ┆ park                         │
│ ---    ┆ ---    ┆ ---                     ┆ ---                          │
│ i64    ┆ str    ┆ str                     ┆ str                          │
╞════════╪════════╪═════════════════════════╪══════════════════════════════╡
│ 1871   ┆ BS1    ┆ Boston Red Stockings    ┆ South End Grounds I          │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ CH1    ┆ Chicago White Stockings ┆ Union Base-Ball Grounds      │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ CL1    ┆ Cleveland Forest Citys  ┆ National Association Grounds │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ FW1    ┆ Fort Wayne Kekiongas    ┆ Hamilton Field               │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1871   ┆ NY2    ┆ New York Mutuals        ┆ Union Grounds (Brooklyn)     │
└────────┴────────┴─────────────────────────┴──────────────────────────────┘

In [ ]:
teams_select.shape

(2925, 4)

## How can we match these columns?

We will need to match on both `teamID` and `yearID` because

1. Team names might change
2. The teams park depends on the year.

## Determining the correct join.

Facts of the task:

* We want batting statistics for all players in 2009
* We want only teams names and parks for teams that played that year.

**Question:** Which join?
**Answer:** `batting >> left_join(teams)` on both year and

## How to join on multiple columns

Pass a list of columns names to `on`

In [ ]:
df_joined = batting_select.join(teams_select, on=['teamID', 'yearID'], how='left')
df_joined.head()

shape: (5, 5)
┌────────┬────────┬─────┬────────────────────────┬─────────────────────────────────────┐
│ teamID ┆ yearID ┆ R   ┆ name                   ┆ park                                │
│ ---    ┆ ---    ┆ --- ┆ ---                    ┆ ---                                 │
│ str    ┆ i64    ┆ i64 ┆ str                    ┆ str                                 │
╞════════╪════════╪═════╪════════════════════════╪═════════════════════════════════════╡
│ TRO    ┆ 1871   ┆ 0   ┆ Troy Haymakers         ┆ Haymakers' Grounds                  │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ RC1    ┆ 1871   ┆ 30  ┆ Rockford Forest Citys  ┆ Agricultural Society Fair Ground... │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ CL1    ┆ 1871   ┆ 28  ┆ Cleveland Forest Citys ┆ National Association Grounds        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ WS3    ┆ 1871   ┆ 28  ┆ Washington Olympics    ┆ Olympics Grounds                    │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ RC1    ┆ 1871   ┆ 29  ┆ Rockford Forest Citys  ┆ Agricultural Society Fair Ground... │
└────────┴────────┴─────┴────────────────────────┴─────────────────────────────────────┘

In [ ]:
df_joined.shape

(107429, 5)

#### Filter, group by, and aggregate to get the desired result.

In [ ]:
runs_per_team09 = (df_joined
                   .filter(pl.col('yearID') == 2009)
                   .groupby([pl.col('name'), pl.col('park')])
                   .agg(pl.col('R').sum().alias('total_runs')))
runs_per_team09.head()

shape: (5, 3)
┌───────────────────────────────┬─────────────────────┬────────────┐
│ name                          ┆ park                ┆ total_runs │
│ ---                           ┆ ---                 ┆ ---        │
│ str                           ┆ str                 ┆ i64        │
╞═══════════════════════════════╪═════════════════════╪════════════╡
│ Seattle Mariners              ┆ Safeco Field        ┆ 640        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Los Angeles Angels of Anaheim ┆ Angel Stadium       ┆ 883        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Los Angeles Dodgers           ┆ Dodger Stadium      ┆ 780        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Chicago White Sox             ┆ U.S. Cellular Field ┆ 724        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Kansas City Royals            ┆ Kauffman Stadium    ┆ 686        │
└───────────────────────────────┴─────────────────────┴────────────┘

In [ ]:
runs_per_team09.shape

(30, 3)

## Joins are expensive!

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`,

* `inner_join` performs `min(len(A), len(B))` comparisons
* `outer_join` performs `len(A)*len(B)` row comparisons.
* `left_join` and `right_join` are somewhere inbetween (depending on how we treat multiples).

## Outer joins (can) produce LARGE tables.

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`,

* `outer_join` might produce `len(A)*len(B)` rows (worst case)
* `left_join` and `right_join` are similar if we don't drop multiples.

#### Last example

In [ ]:
len(batting_select)*len(teams_select)

314229825

## The joining mantra: Filter/aggregate THEN join

By filtering and/or aggregating before joining, you will

* reduce the amount of work
* protect against really large output

#### Filter and aggregate to total runs per team per year

In [ ]:
runs = (batting_select
        .filter(pl.col('yearID') == 2009)
        .groupby(pl.col('teamID'))
        .agg(pl.col('R').sum().alias('total_runs')))

runs.head()

shape: (5, 2)
┌────────┬────────────┐
│ teamID ┆ total_runs │
│ ---    ┆ ---        │
│ str    ┆ i64        │
╞════════╪════════════╡
│ COL    ┆ 804        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ CHA    ┆ 724        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ SLN    ┆ 730        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ NYN    ┆ 671        │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ CIN    ┆ 673        │
└────────┴────────────┘

#### Filter down to 2009 teams

In [ ]:
teams09 = (teams_select
           .filter(pl.col('yearID') == 2009)
          )
teams09.head()

shape: (5, 4)
┌────────┬────────┬──────────────────────┬─────────────────────────────┐
│ yearID ┆ teamID ┆ name                 ┆ park                        │
│ ---    ┆ ---    ┆ ---                  ┆ ---                         │
│ i64    ┆ str    ┆ str                  ┆ str                         │
╞════════╪════════╪══════════════════════╪═════════════════════════════╡
│ 2009   ┆ ARI    ┆ Arizona Diamondbacks ┆ Chase Field                 │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2009   ┆ ATL    ┆ Atlanta Braves       ┆ Turner Field                │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2009   ┆ BAL    ┆ Baltimore Orioles    ┆ Oriole Park at Camden Yards │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2009   ┆ BOS    ┆ Boston Red Sox       ┆ Fenway Park II              │
├╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2009   ┆ CHA    ┆ Chicago White Sox    ┆ U.S. Cellular Field         │
└────────┴────────┴──────────────────────┴─────────────────────────────┘

In [ ]:
len(runs)*len(teams09)

900

In [ ]:
runs_per_team09_new = (runs
                       .join(teams09, on='teamID', how='left')
                       .drop('teamID')
                      )
runs_per_team09_new.head()

shape: (5, 4)
┌────────────┬────────┬─────────────────────┬──────────────────────────┐
│ total_runs ┆ yearID ┆ name                ┆ park                     │
│ ---        ┆ ---    ┆ ---                 ┆ ---                      │
│ i64        ┆ i64    ┆ str                 ┆ str                      │
╞════════════╪════════╪═════════════════════╪══════════════════════════╡
│ 804        ┆ 2009   ┆ Colorado Rockies    ┆ Coors Field              │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 724        ┆ 2009   ┆ Chicago White Sox   ┆ U.S. Cellular Field      │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 730        ┆ 2009   ┆ St. Louis Cardinals ┆ Busch Stadium III        │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 671        ┆ 2009   ┆ New York Mets       ┆ Citi Field               │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 673        ┆ 2009   ┆ Cincinnati Reds     ┆ Great American Ball Park │
└────────────┴────────┴─────────────────────┴──────────────────────────┘

In [ ]:
runs_per_team09_new.shape

(30, 4)

## Don't worry about speed until it matters!

* I don't emphasize speed most of the time.
* In this case, either approach was nearly instant.
* Still ... `join`s are *so dangerous* that I make an exception here!

## <font color="red"> Exercise 7.3.1 </font>

Determine all the players that have hit more than 50 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the `Batting.csv`, `People.csv`, and `Teams.csv` files.  To get credit for this exercise, use the join methods presented above.

In [ ]:
# Your code here